In [1]:
import sys
import os
from dotenv import load_dotenv

from db_connection import load_view

ModuleNotFoundError: No module named 'db_connection'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = load_view("user_metrics")

In [5]:
df.head()

,user_id,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,1,Female,54,CA,Free,26,23,0.20,Desktop,31,False,True
1,2,Other,33,DE,Family,141,62,0.34,Web,0,True,False
2,3,Male,38,AU,Premium,199,38,0.04,Mobile,0,True,True
3,4,Female,22,CA,Student,36,2,0.31,Mobile,0,True,False
4,5,Other,29,US,Family,250,57,0.36,Mobile,0,True,True


Change boolean values to binary (0,1)

In [6]:
df['offline_listening'] = df['offline_listening'].astype(int)

In [8]:
df['is_churned'] = df['is_churned'].astype(int)

In [10]:
df.head()

,user_id,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,1,Female,54,CA,Free,26,23,0.20,Desktop,31,0,1
1,2,Other,33,DE,Family,141,62,0.34,Web,0,1,0
2,3,Male,38,AU,Premium,199,38,0.04,Mobile,0,1,1
3,4,Female,22,CA,Student,36,2,0.31,Mobile,0,1,0
4,5,Other,29,US,Family,250,57,0.36,Mobile,0,1,1


In [14]:
from sklearn.model_selection import train_test_split

Split data: 60% train, 20% validation, 20% test

In [18]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=5)

In [17]:
len(df_train), len(df_val), len(df_test)

(4800, 1600, 1600)

In [19]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [21]:
y_train = df_train.is_churned.values
y_val = df_val.is_churned.values
y_test = df_test.is_churned.values

del df_train['is_churned']
del df_val['is_churned']
del df_test['is_churned']

Calculating global churn rate

In [35]:
global_churn = df_full_train.is_churned.mean()
round(global_churn,2)

np.float64(0.26)

In [24]:
df_full_train.dtypes

user_id                   object
gender                    object
age                        int64
country                   object
subscription_type         object
listening_time             int64
songs_played_per_day       int64
skip_rate                float64
device_type               object
ads_listened_per_week      int64
offline_listening          int64
is_churned                 int64
dtype: object

In [28]:
numerical = ['age', 'listening_time', 'songs_played_per_day', 'skip_rate', 'ads_listened_per_week', 'offline_listening']

In [29]:
categorical = ['gender', 'country', 'subscription_type', 'device_type']

In [30]:
df_full_train[categorical].nunique()

gender               3
country              8
subscription_type    4
device_type          3
dtype: int64

In [31]:
from IPython.display import display

In [36]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).is_churned.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn # difference
    df_group['risk'] = df_group['mean'] / global_churn # risk ratio
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
Female,0.269991,2126,0.009209,1.035314
Male,0.253488,2150,-0.007293,0.972035
Other,0.258945,2124,-0.001836,0.992960




country


,mean,count,diff,risk
country,,,,
AU,0.270677,798,0.009895,1.037945
CA,0.249668,753,-0.011113,0.957385
DE,0.276622,817,0.015841,1.060743
FR,0.283186,791,0.022405,1.085913
IN,0.240444,811,-0.020337,0.922014
PK,0.265664,798,0.004883,1.018724
UK,0.254753,789,-0.006028,0.976883
US,0.245552,843,-0.015230,0.941600




subscription_type


,mean,count,diff,risk
subscription_type,,,,
Family,0.275418,1554,0.014637,1.056128
Free,0.247985,1613,-0.012796,0.950932
Premium,0.254773,1676,-0.006008,0.976962
Student,0.265896,1557,0.005115,1.019613




device_type


,mean,count,diff,risk
device_type,,,,
Desktop,0.261024,2245,0.000243,1.000933
Mobile,0.269175,2073,0.008394,1.032187
Web,0.252161,2082,-0.008620,0.966946


In [37]:
from sklearn.metrics import mutual_info_score

In [40]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.is_churned)

In [41]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

country              0.000548
subscription_type    0.000283
gender               0.000122
device_type          0.000122
dtype: float64

In [43]:
df_full_train[numerical].corrwith(df_full_train.is_churned).abs().sort_values(ascending=False)

skip_rate                0.021850
songs_played_per_day     0.017237
offline_listening        0.016918
listening_time           0.009156
ads_listened_per_week    0.006246
age                      0.004888
dtype: float64

In [44]:
from sklearn.feature_extraction import DictVectorizer

In [45]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [46]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [47]:
def linear_regression(xi):
    result = w0
    
    for j in range(len(w)):
        result = result + xi[j] * w[j]
        
    return result

In [48]:
def logistic_regression(xi):
    score = w0
    
    for j in range(len(w)):
        score = score + xi[j] * w[j]
        
    result = sigmoid(score)
    return result

In [49]:
from sklearn.linear_model import LogisticRegression

In [50]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

/workspaces/spotify-churn-analysis/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [51]:
model.intercept_[0]

np.float64(-0.5270869675634843)

In [52]:
model.coef_[0].round(3)

array([ 0.   ,  0.001, -0.081, -0.076,  0.014,  0.08 , -0.091, -0.17 ,
       -0.081, -0.109, -0.186, -0.109, -0.219, -0.103, -0.196, -0.215,
       -0.   , -0.185,  0.07 ,  0.001, -0.042, -0.329, -0.138, -0.006])

In [53]:
y_pred = model.predict_proba(X_val)[:, 1]

In [54]:
churn_decision = (y_pred >= 0.5)

In [55]:
(y_val == churn_decision).mean()

np.float64(0.7125)

In [56]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = churn_decision.astype(int)
df_pred['actual'] = y_val

In [57]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [58]:
df_pred.correct.mean()

np.float64(0.7125)

In [59]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'ads_listened_per_week': np.float64(0.0),
 'age': np.float64(0.001),
 'country=AU': np.float64(-0.081),
 'country=CA': np.float64(-0.076),
 'country=DE': np.float64(0.014),
 'country=FR': np.float64(0.08),
 'country=IN': np.float64(-0.091),
 'country=PK': np.float64(-0.17),
 'country=UK': np.float64(-0.081),
 'country=US': np.float64(-0.109),
 'device_type=Desktop': np.float64(-0.186),
 'device_type=Mobile': np.float64(-0.109),
 'device_type=Web': np.float64(-0.219),
 'gender=Female': np.float64(-0.103),
 'gender=Male': np.float64(-0.196),
 'gender=Other': np.float64(-0.215),
 'listening_time': np.float64(-0.0),
 'offline_listening': np.float64(-0.185),
 'skip_rate': np.float64(0.07),
 'songs_played_per_day': np.float64(0.001),
 'subscription_type=Family': np.float64(-0.042),
 'subscription_type=Free': np.float64(-0.329),
 'subscription_type=Premium': np.float64(-0.138),
 'subscription_type=Student': np.float64(-0.006)}

In [60]:
small = ['device_type', 'gender', 'subscription_type']

In [61]:
df_train[small].iloc[:10].to_dict(orient='records')

[{'device_type': 'Web', 'gender': 'Male', 'subscription_type': 'Family'},
 {'device_type': 'Desktop', 'gender': 'Female', 'subscription_type': 'Family'},
 {'device_type': 'Web', 'gender': 'Male', 'subscription_type': 'Free'},
 {'device_type': 'Web', 'gender': 'Male', 'subscription_type': 'Student'},
 {'device_type': 'Web', 'gender': 'Female', 'subscription_type': 'Family'},
 {'device_type': 'Mobile', 'gender': 'Female', 'subscription_type': 'Premium'},
 {'device_type': 'Desktop', 'gender': 'Male', 'subscription_type': 'Free'},
 {'device_type': 'Mobile', 'gender': 'Male', 'subscription_type': 'Free'},
 {'device_type': 'Desktop', 'gender': 'Male', 'subscription_type': 'Free'},
 {'device_type': 'Desktop', 'gender': 'Other', 'subscription_type': 'Premium'}]

In [62]:
dicts_train_small = df_train[small].to_dict(orient='records')
dicts_val_small = df_val[small].to_dict(orient='records')

In [63]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,False
,sort,True


In [64]:
dv_small.get_feature_names_out()

array(['device_type=Desktop', 'device_type=Mobile', 'device_type=Web',
       'gender=Female', 'gender=Male', 'gender=Other',
       'subscription_type=Family', 'subscription_type=Free',
       'subscription_type=Premium', 'subscription_type=Student'],
      dtype=object)

In [65]:
X_train_small = dv_small.transform(dicts_train_small)

In [66]:
model_small = LogisticRegression(solver='lbfgs')
model_small.fit(X_train_small, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [67]:
w0 = model_small.intercept_[0]
w0

np.float64(-0.5709920936783696)

In [68]:
w = model_small.coef_[0]
w.round(3)

array([-0.21 , -0.129, -0.226, -0.12 , -0.216, -0.228, -0.109, -0.192,
       -0.188, -0.076])

In [69]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))

{'device_type=Desktop': np.float64(-0.21),
 'device_type=Mobile': np.float64(-0.129),
 'device_type=Web': np.float64(-0.226),
 'gender=Female': np.float64(-0.12),
 'gender=Male': np.float64(-0.216),
 'gender=Other': np.float64(-0.228),
 'subscription_type=Family': np.float64(-0.109),
 'subscription_type=Free': np.float64(-0.192),
 'subscription_type=Premium': np.float64(-0.188),
 'subscription_type=Student': np.float64(-0.076)}

Using the model

In [70]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

In [71]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [73]:
y_full_train = df_full_train.is_churned.values

In [74]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_full_train, y_full_train)

/workspaces/spotify-churn-analysis/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [75]:
dicts_test = df_test[categorical + numerical].to_dict(orient='records')

In [76]:
X_test = dv.transform(dicts_test)

In [77]:
y_pred = model.predict_proba(X_test)[:, 1]

In [78]:
churn_decision = (y_pred >= 0.5)

In [79]:
(churn_decision == y_test).mean()

np.float64(0.74875)

In [80]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [82]:
customer = dicts_test[16]
customer

{'gender': 'Male',
 'country': 'DE',
 'subscription_type': 'Student',
 'device_type': 'Desktop',
 'age': 38,
 'listening_time': 203,
 'songs_played_per_day': 32,
 'skip_rate': 0.08,
 'ads_listened_per_week': 0,
 'offline_listening': 1}

In [83]:
X_small = dv.transform([customer])

In [84]:
model.predict_proba(X_small)[0, 1]

np.float64(0.2644701701925654)

In [86]:
y_test[16]

np.int64(0)